In [1]:
!pip install opencv-python numpy tensorflow keras matplotlib


In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Load Pretrained UNet Model for Segmentation
def load_unet_model(model_path="unet_model.h5"):
    return load_model(model_path)

# Apply Bilateral Filtering (Edge-Preserving)
def apply_bilateral_filter(image):
    return cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

# Apply Adaptive Thresholding
def adaptive_thresholding(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                            cv2.THRESH_BINARY, 11, 2)
    return adaptive_thresh

# Edge-Aware Filtering (Guided Filter)
def guided_filter(image, guided):
    return cv2.ximgproc.guidedFilter(guided, image, radius=8, eps=50)

# Post-Processing: Remove Small Noise Regions (Connected Component Analysis)
def remove_small_regions(mask, min_size=500):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    new_mask = np.zeros_like(mask)
    
    for i in range(1, num_labels):  # Ignore background
        if stats[i, cv2.CC_STAT_AREA] > min_size:
            new_mask[labels == i] = 255
            
    return new_mask

# Main Pipeline
def process_underwater_image(image_path, model_path="unet_model.h5"):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 256))

    # Step 1: Noise Reduction
    filtered_image = apply_bilateral_filter(image)
    thresholded_image = adaptive_thresholding(filtered_image)

    # Step 2: Deep Learning-Based Segmentation
    model = load_unet_model(model_path)
    input_image = np.expand_dims(filtered_image / 255.0, axis=0)  # Normalize & expand dims
    segmented_mask = model.predict(input_image)[0, :, :, 0]  # Get segmentation output
    segmented_mask = (segmented_mask > 0.5).astype(np.uint8) * 255  # Threshold

    # Step 3: Post-processing
    refined_mask = guided_filter(segmented_mask, filtered_image)
    final_mask = remove_small_regions(refined_mask)

    # Display Results
    fig, axes = plt.subplots(1, 4, figsize=(12, 5))
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)); axes[0].set_title("Original")
    axes[1].imshow(thresholded_image, cmap="gray"); axes[1].set_title("Adaptive Thresholding")
    axes[2].imshow(segmented_mask, cmap="gray"); axes[2].set_title("UNet Segmentation")
    axes[3].imshow(final_mask, cmap="gray"); axes[3].set_title("Post-processed Mask")
    for ax in axes: ax.axis("off")
    plt.show()

# Example Usage (Run in a new cell)
process_underwater_image("sample_underwater.jpg", "unet_model.h5")


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
